In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime

In [2]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 04A9-8968

 C:\Users\나\Desktop\업무\Data Analysis 디렉터리

2020-12-11  오전 10:15    <DIR>          .
2020-12-11  오전 10:15    <DIR>          ..
2020-12-11  오전 10:15    <DIR>          .ipynb_checkpoints
2020-09-26  오전 05:40     3,248,970,693 AL_11_D151_20200925.csv
2020-12-07  오후 03:32           882,655 coor_df.csv
2020-11-21  오후 07:52               315 debug.log
2020-12-03  오후 03:45           245,179 EDA.ipynb
2020-11-24  오후 03:05        87,232,378 entrc_seoul.txt
2020-12-11  오전 10:15                72 Feature Engineering 2.ipynb
2020-12-11  오전 10:14           448,823 Feature Engineering.ipynb
2020-12-04  오후 04:45         5,050,450 get_coor.csv
2020-12-07  오후 12:52               115 juso_api.txt
2020-12-04  오후 05:45                43 kakao_api.txt
2020-12-11  오전 10:14         7,231,071 original_coor_data_df.csv
2020-12-02  오후 05:00           348,527 Practice.ipynb
2020-12-02  오전 10:56           432,892 서울_오피스텔_2020_01_01_2020_10_31.ipynb
2020-12-02  오후 02:43

# Read Data

In [3]:
def land_number_allocator(x):
    try:
        return str(int(x))
    except:
        return x

In [4]:
def full_address_allocator(df):
    
    df['전체주소'] = '서울특별시' + ' ' + df['자치구명'] + ' ' + df['법정동명'] + ' '\
                    + df['지번코드'].apply(str).str[-8:-4].apply(land_number_allocator) + '-'\
                    + df['지번코드'].apply(str).str[-4:].apply(land_number_allocator)
    
    
    simple_cols = ['전체주소', '신고년도', '건물면적', '층정보', '물건금액', '건축년도', '건물주용도']
    
    return df[simple_cols].dropna().drop_duplicates()

In [7]:
def create_transaction_info_df():
    basedir = './서울시 부동산 실거래가 정보/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.csv'):
            try:
                df = pd.read_csv(basedir + filename, low_memory=False)
            except:
                df = pd.read_csv(basedir + filename, encoding='euc-kr', low_memory=False)
            #officetel_df = df[df['건물주용도'] == '오피스텔']
            #officetel_df['물건금액'] = officetel_df['물건금액']
            
            dfs_list.append(full_address_allocator(df))
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [31]:
transaction_info_df = create_transaction_info_df()
transaction_info_df.shape

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:14<00:00,  1.02it/s]


(1907507, 7)

In [32]:
transaction_info_df.head()

,전체주소,신고년도,건물면적,층정보,물건금액,건축년도,건물주용도
0,서울특별시 양천구 신정동 339-0,2006,59.83,4.0,215000000,2001.0,아파트
1,서울특별시 강동구 천호동 214-60,2006,50.25,1.0,135000000,1990.0,연립주택
2,서울특별시 양천구 목동 709-2,2006,77.17,14.0,273000000,2000.0,아파트
3,서울특별시 강동구 길동 160-0,2006,111.51,2.0,425000000,1983.0,아파트
4,서울특별시 양천구 신정동 314-0,2006,71.77,10.0,630000000,1987.0,아파트


In [33]:
transaction_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1907507 entries, 0 to 1907506
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   전체주소    object 
 1   신고년도    int64  
 2   건물면적    float64
 3   층정보     float64
 4   물건금액    int64  
 5   건축년도    float64
 6   건물주용도   object 
dtypes: float64(3), int64(2), object(2)
memory usage: 101.9+ MB


In [9]:
official_land_prices_df = pd.read_csv('AL_11_D151_20200925.csv', encoding='euc-kr', usecols=[2,5,6,8])
official_land_prices_df.shape

(29393777, 4)

In [11]:
official_land_prices_df.head()

,법정동명,지번,기준년도,공시지가
0,서울특별시 종로구 청운동,1,1990,750000
1,서울특별시 종로구 청운동,1,1991,980000
2,서울특별시 종로구 청운동,1,1992,980000
3,서울특별시 종로구 청운동,1,1993,980000
4,서울특별시 종로구 청운동,1,1994,960000


In [23]:
official_land_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29393777 entries, 0 to 29393776
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   법정동명    object
 1   지번      object
 2   기준년도    int64 
 3   공시지가    int64 
dtypes: int64(2), object(2)
memory usage: 897.0+ MB


In [13]:
land_plans_df = pd.read_csv('./토지이용계획정보/AL_11_D155_20201107.csv', encoding='euc-kr', low_memory=False).drop_duplicates()
land_plans_df.shape

(8455920, 13)

In [14]:
land_plans_df.head()

,고유번호,법정동코드,법정동명,대장구분코드,대장구분명,지번,도면번호,저촉여부코드,저촉여부,용도지역지구코드,용도지역지구명,등록일자,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,12900001111020080001UNE2010001001,1,포함,UNE201,대공방어협조구역(위탁고도:54-236m),2009-12-29,2020-06-09
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,13404001111020080001UOA1200010002,2,저촉,UOA120,상대보호구역,2009-12-28,2020-06-09
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQA01X0000001,1,포함,UQA01X,도시지역,2009-12-28,2020-06-09
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQF1100071000,2,저촉,UQF110,자연경관지구,2009-12-28,2020-06-09
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,토지대장,1,15000001111020000000UQS1000507000,2,저촉,UQS100,도로,2009-12-28,2020-06-09


In [24]:
land_plans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8455920 entries, 0 to 8455919
Data columns (total 13 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고유번호      int64 
 1   법정동코드     int64 
 2   법정동명      object
 3   대장구분코드    int64 
 4   대장구분명     object
 5   지번        object
 6   도면번호      object
 7   저촉여부코드    int64 
 8   저촉여부      object
 9   용도지역지구코드  object
 10  용도지역지구명   object
 11  등록일자      object
 12  데이터기준일자   object
dtypes: int64(4), object(9)
memory usage: 903.2+ MB


In [15]:
def add_0(x):
    splitted = x.split('-')
    if len(splitted) == 1:
        return x+'-0'
    else:
        return x

In [16]:
def read_coor_data():
    basedir = './좌표데이터3/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.csv'):
            df = pd.read_csv(basedir + filename)
            dfs_list.append(df)
            
    concat_df = pd.concat(dfs_list).reset_index(drop=True).drop_duplicates()
    
    concat_df = concat_df[['전체주소', 'x좌표', 'y좌표']]
    
    concat_df['전체주소'] = concat_df['전체주소'].apply(add_0)
    
    return concat_df

In [19]:
coor_df = pd.read_csv('coor_data_df.csv', index_col=0)
coor_df.shape

(84349, 3)

In [20]:
coor_df.head()

,전체주소,x좌표,y좌표
0,서울특별시 양천구 신정동 339-0,944806.276643,1.947001e+06
1,서울특별시 강동구 길동 160-0,968954.010468,1.949060e+06
2,서울특별시 양천구 신정동 314-0,944212.602398,1.947199e+06
3,서울특별시 강동구 고덕동 212-0,970202.844451,1.950847e+06
4,서울특별시 양천구 신정동 329-0,944259.661105,1.946646e+06


In [21]:
coor_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84349 entries, 0 to 84348
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   전체주소    84349 non-null  object 
 1   x좌표     84349 non-null  float64
 2   y좌표     84349 non-null  float64
dtypes: float64(2), object(1)
memory usage: 2.6+ MB


# Preprocessing

In [59]:
def transaction_info_preprocessing(df):
    df = df.copy()
    
    df['층정보'] = df['층정보'].astype(int)
    df['건축년도'] = df['건축년도'].astype(int)
    
    #simple_cols = ['전체주소', '신고년도', '건물면적', '층정보', '물건금액', '건축년도', '건물주용도']
    #df.columns = ['전체주소', '년', '전용면적(㎡)', '층', '가격', '신고년도', '건축년도', '건물종류']
    
    '''df['아파트'] = 0
    df['연립주택'] = 0
    df['오피스텔'] = 0
    
    df['아파트'] = df['건물종류'].apply(lambda x: 1 if x == '아파트' else 0)
    df['연립주택'] = df['건물종류'].apply(lambda x: 1 if x == '연립주택' else 0)
    df['오피스텔'] = df['건물종류'].apply(lambda x: 1 if x == '오피스텔' else 0)
    
    simple_cols = ['전체주소', '년', '전용면적(㎡)', '층', '가격', '신고년도', '건축년도', '아파트', '연립주택', '오피스텔']
    
    return df[simple_cols]'''
    
    return df

In [60]:
processed_transaction_info_df = transaction_info_preprocessing(transaction_info_df)
processed_transaction_info_df.shape

(1907507, 7)

In [61]:
processed_transaction_info_df.head()

,전체주소,신고년도,건물면적,층정보,물건금액,건축년도,건물주용도
0,서울특별시 양천구 신정동 339-0,2006,59.83,4,215000000,2001,아파트
1,서울특별시 강동구 천호동 214-60,2006,50.25,1,135000000,1990,연립주택
2,서울특별시 양천구 목동 709-2,2006,77.17,14,273000000,2000,아파트
3,서울특별시 강동구 길동 160-0,2006,111.51,2,425000000,1983,아파트
4,서울특별시 양천구 신정동 314-0,2006,71.77,10,630000000,1987,아파트


In [63]:
processed_transaction_info_df['층정보'].value_counts()

 2     247355
 3     227647
 4     198240
 1     171385
 5     147897
 6      95608
 7      85389
 8      79349
 9      76014
 10     73455
 11     68271
 12     64709
 13     57303
-1      53935
 14     52971
 15     47962
 16     26416
 17     24531
 18     21820
 19     18203
 20     15017
 21     10639
 22      9077
 23      7568
 24      5863
 25      4555
 26      2727
 27      2398
 28      1879
 29      1441
 30      1206
 31       972
 32       813
 33       732
 34       664
 35       597
 36       447
 37       296
 38       255
 0        250
 39       237
 40       167
 41       143
 42       126
 43       123
 44       102
 46        91
-2         81
 45        81
 47        73
 48        61
 51        42
 49        38
 54        37
 50        31
 52        25
 53        21
 55        16
 57        16
 58        14
-3         12
 59        11
 56        10
 63        10
 66         9
 71         9
 72         8
 62         8
 64         8
 68         7
 60         6
 65   

In [64]:
floors_80_90_df = processed_transaction_info_df[processed_transaction_info_df['층정보'] >= 80]
floors_80_90_df.shape

(2, 7)

In [65]:
floors_80_90_df

,전체주소,신고년도,건물면적,층정보,물건금액,건축년도,건물주용도
890743,서울특별시 마포구 상수동 205-0,2013,84.92,80,624200000,0,아파트
1189998,서울특별시 송파구 장지동 894-0,2015,100.85,90,744140000,0,아파트


In [66]:
floors_over_60_df = processed_transaction_info_df[processed_transaction_info_df['층정보'] >= 60]
floors_over_60_df.shape

(90, 7)

In [53]:
floors_over_60_df.head(50)

,전체주소,년,전용면적(㎡),층,가격,건축년도,건물나이,아파트,연립주택,오피스텔
4361,서울특별시 양천구 목동 916-0,2006,167.44,66,1960000000,2003,3,1,0,0
95374,서울특별시 강남구 도곡동 467-29,2006,163.94,65,4280000000,2004,2,1,0,0
97344,서울특별시 강남구 도곡동 467-29,2006,141.71,63,2060000000,2004,2,1,0,0
100479,서울특별시 강남구 도곡동 467-29,2006,244.22,62,5000000000,2004,2,1,0,0
228655,서울특별시 양천구 목동 916-0,2007,138.20,63,1550000000,2003,4,1,0,0
297115,서울특별시 강남구 도곡동 467-29,2007,235.74,68,4320000000,2004,3,1,0,0
449121,서울특별시 강남구 도곡동 467-29,2009,214.97,62,1650000000,2004,5,1,0,0
449349,서울특별시 강남구 도곡동 467-29,2009,214.97,66,4500000000,2004,5,1,0,0
470050,서울특별시 양천구 목동 916-0,2009,167.44,60,2450000000,2003,6,1,0,0
474580,서울특별시 양천구 목동 916-0,2009,154.05,67,1780000000,2003,6,1,0,0


In [67]:
floors_over_70_df = processed_transaction_info_df[processed_transaction_info_df['층정보'] >= 70]
floors_over_70_df.shape

(25, 7)

In [71]:
floors_over_70_df

,전체주소,신고년도,건물면적,층정보,물건금액,건축년도,건물주용도
890743,서울특별시 마포구 상수동 205-0,2013,84.92,80,624200000,0,아파트
1189998,서울특별시 송파구 장지동 894-0,2015,100.85,90,744140000,0,아파트
1251936,서울특별시 동작구 흑석동 158-1,2016,84.91,73,850440000,0,아파트
1275226,서울특별시 동작구 흑석동 158-1,2016,84.91,73,691000000,0,아파트
1298452,서울특별시 성동구 옥수동 528-0,2016,107.99,71,915120000,0,아파트
1411857,서울특별시 동작구 흑석동 158-1,2017,84.91,71,792340000,0,아파트
1411861,서울특별시 동작구 흑석동 158-1,2017,84.91,72,758890000,0,아파트
1411909,서울특별시 동작구 흑석동 158-1,2017,84.91,71,756640000,0,아파트
1411911,서울특별시 동작구 흑석동 158-1,2017,84.91,72,787440000,0,아파트
1411920,서울특별시 동작구 흑석동 158-1,2017,84.91,72,782440000,0,아파트


In [72]:
jangji_df = processed_transaction_info_df[processed_transaction_info_df['전체주소'] == '서울특별시 송파구 장지동 894-0']
jangji_df.shape

(154, 7)

In [74]:
jangji_df.head(50)

,전체주소,신고년도,건물면적,층정보,물건금액,건축년도,건물주용도
1017399,서울특별시 송파구 장지동 894-0,2014,87.88,17,624760000,0,아파트
1017569,서울특별시 송파구 장지동 894-0,2014,87.68,18,652410000,0,아파트
1017729,서울특별시 송파구 장지동 894-0,2014,114.95,20,863690000,0,아파트
1017733,서울특별시 송파구 장지동 894-0,2014,87.83,7,647460000,0,아파트
1017771,서울특별시 송파구 장지동 894-0,2014,100.78,8,758470000,0,아파트
1017990,서울특별시 송파구 장지동 894-0,2014,87.98,11,648120000,0,아파트
1018207,서울특별시 송파구 장지동 894-0,2014,87.96,6,637430000,0,아파트
1018211,서울특별시 송파구 장지동 894-0,2014,87.88,8,664760000,0,아파트
1018401,서울특별시 송파구 장지동 894-0,2014,87.96,15,590390000,0,아파트
1024106,서울특별시 송파구 장지동 894-0,2014,128.83,7,598500000,0,아파트


In [ ]:
def bon_bun_allocator(x):
    splitted = x.split('-')
    return int(splitted[0])

def bu_bun_allocator(x):
    splitted = x.split('-')
    if len(splitted) == 2:
        return int(splitted[1])
    elif len(splitted) == 1:
        return 0
    else:
        print('error')
        return

In [ ]:
def create_land_prices_df():
    df = pd.read_csv('AL_11_D151_20200925.csv', encoding='euc-kr', usecols=[2,5,6,8])
    
    since_2006_df = df[df['기준년도'] >= 2006]
    
    simple_cols = ['법정동명', '지번', '기준년도', '공시지가']
    
    bonbun = df['지번'].apply(bon_bun_allocator)
    bubun = df['지번'].apply(bu_bun_allocator)
    df['전체주소'] = df['법정동명'] + ' ' + bonbun.apply(str) + '-' + bubun.apply(str)
    
    df = df[['전체주소', '기준년도', '공시지가']]
    
    df['공시지가'] = df['공시지가'] / 10000
    
    df.columns = ['전체주소', '년', '공시지가(만원)']
    
    return df

In [ ]:
def create_secondary_officetel_prices_df():
    basedir = './오피스텔 실거래가/'
    filenames = os.listdir(basedir)
    
    dfs_list = []
    
    for filename in tqdm(filenames, position=0):
        if filename.endswith('.xlsx'):
            df = pd.read_excel(basedir + filename, header=16).drop_duplicates()
            
            df['전체주소'] = df['시군구'] + ' ' + df['번지']
            
            df['신고년도'] = df['계약년월'].apply(str).str[:4]
            df['월'] = df['계약년월'].apply(str).str[4:]
            
            df['거래금액(만원)'] = df['거래금액(만원)'].str.replace(',', '').apply(float)
            
            
            simple_cols = ['전체주소', '전용면적(㎡)', '신고년도', '월', '거래금액(만원)', '층']
            
            dfs_list.append(df[simple_cols].dropna())
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    concat_df['신고년도'] = concat_df['신고년도'].astype(int)
    
    concat_df.columns = ['전체주소', '전용면적(㎡)', '년', '월', '가격(만원)', '층']
      
    return concat_df

In [ ]:
def land_plans_full_address_allocator(df):
    bonbun = df['지번'].apply(bon_bun_allocator)
    bubun = df['지번'].apply(bu_bun_allocator)
    
    df['전체주소'] = df['법정동명'] + ' ' + bonbun.apply(str) + '-' + bubun.apply(str)
    
    simple_cols = ['전체주소', '용도지역지구명']
    df = df[simple_cols]
    
    return df.dropna().drop_duplicates()

In [ ]:
def land_plan_unstack(df):
    
    df['이진항'] = 1
    df = df.set_index(['전체주소', '용도지역지구명'])
    df = df.unstack(-1, 0).reset_index()
    
    cols = df.columns
    new_cols = []
    for i in range(len(cols)):
        if i == 0:
            new_cols.append(cols[i][0])
        else:
            new_cols.append(cols[i][1])
    
    df.columns = new_cols
    
    return df

In [ ]:
def create_land_plans_df():    
    df = land_plans_full_address_allocator(df)    
    df = land_plan_unstack(df)
    
    return df

In [ ]:
def fetch_coor_data(df):
    unique_addresses = df['전체주소'].unique()
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIwMTIwNzEyNTE1OTExMDUxMTk='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIwMTIwNDE3NDIyNDExMDUwNTc='
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    addr_list = []
    coor_x_list = []
    coor_y_list = []
    
    count = 0
    
    for address in tqdm(unique_addresses, position=0):        
        try:
            roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                           + address + '&resultType=json' + '&hstryYn=Y')
            roadname_info = roadname_result.json()['results']['juso'][0]

            coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd'] + '&rnMgtSn='\
                                       + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn'] + '&buldMnnm='\
                                       + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno'] +'&resultType=json')
            coor_info = coor_result.json()['results']['juso'][0]

            coor_x = pd.to_numeric(coor_info['entX'])
            coor_y = pd.to_numeric(coor_info['entY'])

            addr_list.append(address)
            coor_x_list.append(coor_x)
            coor_y_list.append(coor_y)
        except:
            if address.endswith('-1'):
                pass
            else:
                try:
                    address = address.replace('-0', '')
                    
                    roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' 
                                                   + '&keyword=' + address + '&resultType=json' + '&hstryYn=Y')
                    roadname_info = roadname_result.json()['results']['juso'][0]

                    coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd'] + '&rnMgtSn='\
                                               + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn'] + '&buldMnnm='\
                                               + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno'] +'&resultType=json')
                    coor_info = coor_result.json()['results']['juso'][0]

                    coor_x = pd.to_numeric(coor_info['entX'])
                    coor_y = pd.to_numeric(coor_info['entY'])

                    addr_list.append(address)
                    coor_x_list.append(coor_x)
                    coor_y_list.append(coor_y)

                except:
                    pass
            
        
        count += 1
        
        if (count % np.floor(len(unique_addresses)/10)) == 0:
            coor_dict = {'전체주소': addr_list, 'x좌표': coor_x_list, 'y좌표': coor_y_list}
            coor_df = pd.DataFrame(coor_dict)
            coor_df.to_csv('original_coordinates_data{}.csv'.format(datetime.datetime.now()))
            
            
        
    coor_dict = {'전체주소': addr_list, 'x좌표': coor_x_list, 'y좌표': coor_y_list}
    
    coor_df = pd.DataFrame(coor_dict)
    
    return coor_df

In [ ]:
def coordinates_feature_engineering(df):
    x_med = 951489.8566332166 # 가장 처음 조사했던 x 좌표 자료의 중간값(median).
    y_med = 1948900.6749279741 # 가장 처음 조사했던 y 좌표 자료의 중간값(median).
    x_std = 7755.838697 # 가장 처음 조사했던 x 좌표 자료의 표준편차.
    y_std = 5136.491358 # 가장 처음 조사했던 y 좌표 자료의 표준편차.
    
    df['x_norm'] = (df['x좌표'] - x_med) /1000 #/ x_std
    df['xdegree2'] = df['x_norm']**2 / 2
    df['xdegree3'] = df['x_norm']**3 / 6
    df['xdegree4'] = df['x_norm']**4 / math.factorial(4)
    df['xdegree5'] = df['x_norm']**5 / math.factorial(5)
    df['xdegree6'] = df['x_norm']**6 / math.factorial(6)
    df['xdegree7'] = df['x_norm']**7 / math.factorial(7)
    df['xdegree8'] = df['x_norm']**8 / math.factorial(8)
    df['xdegree9'] = df['x_norm']**9 / math.factorial(9)
    df['xdegree10'] = df['x_norm']**10 / math.factorial(10)
    df['xdegree11'] = df['x_norm']**11 / math.factorial(11)
    df['xdegree12'] = df['x_norm']**12 / math.factorial(12)
    df['xdegree13'] = df['x_norm']**13 / math.factorial(13)
    df['xdegree14'] = df['x_norm']**14 / math.factorial(14)
    df['xdegree15'] = df['x_norm']**15 / math.factorial(15)
    df['xdegree16'] = df['x_norm']**16 / math.factorial(16)
    df['xdegree17'] = df['x_norm']**17 / math.factorial(17)
    df['xdegree18'] = df['x_norm']**18 / math.factorial(18)
    df['xdegree19'] = df['x_norm']**19 / math.factorial(19)
    df['xdegree20'] = df['x_norm']**20 / math.factorial(20)
    df['sin(x_norm)'] = df['x_norm'].apply(math.sin)
    df['cos(x_norm)'] = df['x_norm'].apply(math.cos)
    df['tan(x_norm)'] = df['x_norm'].apply(math.tan)
    
    df['y_norm'] = (df['y좌표'] - y_med) /1000 #/ y_std
    df['ydegree2'] = df['y_norm']**2 / 2
    df['ydegree3'] = df['y_norm']**3 / 6
    df['ydegree4'] = df['y_norm']**4 / math.factorial(4)
    df['ydegree5'] = df['y_norm']**5 / math.factorial(5)
    df['ydegree6'] = df['y_norm']**6 / math.factorial(6)
    df['ydegree7'] = df['y_norm']**7 / math.factorial(7)
    df['ydegree8'] = df['y_norm']**8 / math.factorial(8)
    df['ydegree9'] = df['y_norm']**9 / math.factorial(9)
    df['ydegree10'] = df['y_norm']**10 / math.factorial(10)
    df['ydegree11'] = df['y_norm']**11 / math.factorial(11)
    df['ydegree12'] = df['y_norm']**12 / math.factorial(12)
    df['ydegree13'] = df['y_norm']**13 / math.factorial(13)
    df['ydegree14'] = df['y_norm']**14 / math.factorial(14)
    df['ydegree15'] = df['y_norm']**15 / math.factorial(15)
    df['ydegree16'] = df['y_norm']**16 / math.factorial(16)
    df['ydegree17'] = df['y_norm']**17 / math.factorial(17)
    df['ydegree18'] = df['y_norm']**18 / math.factorial(18)
    df['ydegree19'] = df['y_norm']**19 / math.factorial(19)
    df['ydegree20'] = df['y_norm']**20 / math.factorial(20)
    df['sin(y_norm)'] = df['y_norm'].apply(math.sin)
    df['cos(y_norm)'] = df['y_norm'].apply(math.cos)
    df['tan(y_norm)'] = df['y_norm'].apply(math.tan)
    
    cols = ['전체주소', 'x_norm',
            'xdegree2', 'xdegree3', 'xdegree4', 'xdegree5',
            'xdegree6', 'xdegree7', 'xdegree8', 'xdegree9', 'xdegree10', 
            'xdegree11', 'xdegree12', 'xdegree13', 'xdegree14', 'xdegree15',
            'xdegree16', 'xdegree17', 'xdegree18', 'xdegree19', 'xdegree20',
            'sin(x_norm)', 'cos(x_norm)', 'tan(x_norm)',
            'y_norm',
            'ydegree2', 'ydegree3', 'ydegree4', 'ydegree5',
            'ydegree6', 'ydegree7', 'ydegree8', 'ydegree9', 'ydegree10',
            'ydegree11', 'ydegree12', 'ydegree13', 'ydegree14', 'ydegree15',
            'ydegree16', 'ydegree17', 'ydegree18', 'ydegree19', 'ydegree20',
            'sin(y_norm)', 'cos(y_norm)', 'tan(y_norm)']
    
    return df[cols]

In [ ]:
def month_circular_sine_func(x):
    return math.sin(2*math.pi*((x-1)/12))

In [ ]:
def month_circular_cosine_func(x):
    return math.cos(2*math.pi*((x-1)/12))

In [ ]:
def time_feature_engineering(df):
    df['time_linear'] = df['년'] - df['년'].min() + 1
    #df['month_circular_sine'] = df['월'].apply(float).apply(month_circular_sine_func)
    #df['month_circular_cosine'] = df['월'].apply(float).apply(month_circular_cosine_func)
    
    return df